<a href="https://colab.research.google.com/github/Hectooooooor/AI/blob/main/2.0702_Colab_LINE_Bot_with_GEMINI_Stateful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [9]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token.strip()}", # Added .strip() here
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://liz-magnetostrictive-discomposingly.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://liz-magnetostrictive-discomposingly.ngrok-free.dev


True

In [10]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [11]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [12]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Ming Hsin University of Science and Technology, MHUST）是一所位於台灣新竹縣新豐鄉，歷史悠久且以務實致用為導向的私立科技大學。

**主要特點如下：**

1.  **歷史悠久與地位：**
    *   創立於1966年，前身為「明新工業專科學校」，是台灣第一所私立工業專科學校，在技職教育領域佔有重要地位。
    *   於2002年改制為明新科技大學。

2.  **地理位置優勢：**
    *   位於新竹縣，鄰近新竹科學園區及周邊高科技產業聚落，享有得天獨厚的地理優勢，有利於產學合作與學生實習就業。

3.  **務實致用導向：**
    *   學校以「敬業樂群、研究創新」為校訓，秉持「學以致用」的精神，致力於培養具備專業技能與實務操作能力的現代科技人才。
    *   特別強調實務教學與產學合作，課程設計緊密結合產業需求，縮短學用落差。

4.  **學院與學制：**
    *   目前設有：
        *   **工程學院**
        *   **管理學院**
        *   **服務產業學院**
        *   **設計學院**
    *   提供學士班、碩士班等不同層次的學位課程。

5.  **就業競爭力：**
    *   由於其深厚的產學連結和強調實務能力的教學特色，明新科大畢業生在就業市場上具有良好的競爭力，就業率表現優異。

總體而言，明新科技大學是一所深耕台灣技職教育超過半世紀，以務實創新為核心價值，旨在為國家社會培育高素質專業人才的重要學府。


In [14]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長是 **劉國偉 教授**。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Jan/2026 13:37:39] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[]}


INFO:werkzeug:127.0.0.1 - - [07/Jan/2026 13:49:15] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[]}


...

ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1799486110.py", line 39, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.12/di

BODY:  {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[{"type":"message","message":{"type":"text","id":"595531125716680842","quoteToken":"8pE_E3nUW8kw52vUaJ8ncfqHZfk_xSwzrTk4YcrvqC9OiLQ0TSXZ6I4RbQk4YviVnVH5Yvu5D2XPieB8iY36RDaXp7CaKDEWZS2mb0wpjc8aYHrmEK60wSBsCi_iTsG97XVzGoC9X_vpzpYixSf5Qw","markAsReadToken":"wY0Fkb3MLO4-ZaQHXtwwlnaWLJ-2eA5gy-ZIhYlEZxQIhHgsBMBeNX4V6ffJZi0cr3VxF1I0caOVjV6UB3Bjd24XkZ-w-I9w9jdTR72uSBwMa8TNG7W3QF6v36fzcJXs8ZcTGFoTmZDJkuQoUugZwhLOorPJBifi7eQIoY2vr_15KUtazJ-ncI-L-KF1RrtEDg8B9f0g720bQSDp_Y5gNw","text":"AI 介紹明新科大"},"webhookEventId":"01KECD5RPT2D9FPE7Q8HQJ4N16","deliveryContext":{"isRedelivery":false},"timestamp":1767795516042,"source":{"type":"user","userId":"Ue020f480d3b30476c7126cc8cf6811c0"},"replyToken":"3709f25797bd4005a3165dc58aa1a955","mode":"active"}]}


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1799486110.py", line 39, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.12/di

BODY:  {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[{"type":"message","message":{"type":"text","id":"595531192708104635","quoteToken":"J2GQhxjngAr-ue82jBPXkJqjaT74Iu-4BEp6XJQiqXWHp5A_LPvb2EJW47tMwZ-HoY81YtGBeMaOaNRGXrxWug2oYHPvJme_cLBqgTL2n-PXxejYpwTZwUXhV8ch94Y86SXEfavbAmbdR-TyPNJCIw","markAsReadToken":"k2oLuDUzhsrCvGikXSXFXdkxH6f3vBzRKgotMzC9EuNMvlWKeAaImx-KKT2MD1lt_uvCN6waJXewanQXr-_LGWH2GNPPt5-u3kTzdoMxIQb_Q3gGS1CjAf8d4VP7WPYBTaci3pthlcIlr30Fknj87XmMCDjt4eZ-tGmHLsF8ZtZzjqqAMaL8CedUGa2lme4JNjcpH47Hn503ydZ2G39uQQ","text":"AI 校長是誰"},"webhookEventId":"01KECD702Y116WNNA2H28E1EB6","deliveryContext":{"isRedelivery":false},"timestamp":1767795555952,"source":{"type":"user","userId":"Ue020f480d3b30476c7126cc8cf6811c0"},"replyToken":"5102032540754d94bcbfc6f5f1932c89","mode":"active"}]}


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1799486110.py", line 39, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.12/di